<a href="https://colab.research.google.com/github/moreirapontocom/imersao-ia-alura-google/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [175]:
!pip install -U -q google-generativeai

In [ ]:
import google.generativeai as genai
import requests as req
import json
import random
import textwrap
from IPython.display import display, Markdown
from google.colab import userdata

API_ENDPOINT = userdata.get('API_ENDPOINT')
API_KEY = userdata.get('API_KEY')
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("models/gemini-pro")

model.start_chat()

In [179]:
# Lista todas as mensagens já salvas no banco de dados
# e adiciona na variável "mensagens"

def getMensagens():
  response = req.get(url=f"{API_ENDPOINT}/alura-desafio/mensagens")
  if response.status_code != 200:
    return []

  return response.json()['messages']

mensagens = getMensagens()
totalMensagens = len(mensagens)

In [181]:
# Pega uma mensagem aleatória e exibe na tela.

mensagens = getMensagens()
random_index = random.randint(0, len(mensagens)-1)
random_message = mensagens[random_index]

display(Markdown(f"![Recebam nossos abraços](https://cdn-icons-png.flaticon.com/256/1361/1361323.png "")"))

display(Markdown(f"Mensagem de paz"))
display(Markdown(f"#\"{random_message['message']}\""))


![Recebam nossos abraços](https://cdn-icons-png.flaticon.com/256/1361/1361323.png )

Mensagem de paz

#"Espero que todos possam se recuperar desta tragédia e que Deus traga muito conforto àqueles que perderam amigos e familiares."

In [159]:
# Solicita que o usuário escreva uma mensagem
# Adiciona o input do usuário em "user_message"

print("Escreva uma mensagem de conforto para as pessoas afetadas pelas chuvas no RS")
user_message = input()

Escreva uma mensagem de conforto para as pessoas afetadas pelas chuvas no RS
Desejamos que todos recuperem os bens perdidos nesta tragédia e que Deus traga conforto àqueles que perderam seus entes queridos.


In [176]:
# Concatena o meu prompt com a mensagem que o usuário escreveu
# para fazer a solicitação da análise e adicionar alguns filtros
# Estou solicitando a resposta num formato específico para decidir salvar
# ou não no banco de dados dependendo do resultado da análise pelo modelo.

message_to_analyze = """Analise este texto para detectar algum tipo de abuso,
  discurso de ódio, xingamentos ou qualquer coisa ofensiva.
  Me informe também, em uma única palavra, o resultado da análise.
  Me responda APENAS com "ofensivo" ou "nao_ofensivo".
  Analise este texto"""

# Envia a solicitação de análise para o model

response = model.generate_content(f'{message_to_analyze}:{user_message}')
responseText = response.text if response.text else '{message: "error"}'

print(responseText)

nao_ofensivo


In [177]:
# Definição da função que vai salvar a frase (input) do usuário no banco de dados

def saveMessage(modelConclusion):
  data = { "message": user_message, "model_conclusion": modelConclusion }
  req.post(url=f"{API_ENDPOINT}/alura-desafio/mensagens", json=data)
  return

finalMessage = ""

# Se o modelo detectou o conteúdo (input do usuário) como "nao_ofensivo",
# então a frase será salva no banco de dados (saveMessage()) e também solicito uma nova
# mensagem para o modelo, pedindo um agradecimento.
# Se não - conteúdo detectado como "ofensivo" - o modelo deverá
# responder que a frase não será salva.

if responseText == "nao_ofensivo":
  saveMessage(responseText)
  finalMessage = """Escreva uma mensagem, de no máximo 2 linhas, de
    agradecimento ao usuário por ter enviado uma bela mensagem.
    Escreva em nome de um grupo de pessoas agradecidas.
    Não adicione o nome do usuário.
    Informe também que a mensagem foi salva e será enviada para aqueles
    que precisam ouvir uma palavra de esperança."""
else:
  finalMessage = """Informe ao usuário que a mensagem não será
    publicada pois foi considerada ofensiva. Diga também que este é um
    momento delicado e que devemos ter empatia pelo próximo."""

finalMessageContent = model.generate_content(finalMessage).text

display(Markdown(f"**{finalMessageContent}**"))

**Gratos pela sua mensagem inspiradora que encheu nossos corações de alegria. Ela será compartilhada com aqueles que necessitam de palavras de esperança.**